# Register SmartSPIM Data To CCFv3.1 Mouse Brain Atlas

Atlas reference: http://help.brain-map.org/display/mouseconnectivity/API

The CCF v3.1 atlas with updated spacing and spatial orientation is available at http://download.alleninstitute.org/informatics-archive/converted_mouse_ccf/average_template/.

Refer to [Get-S3-Data](Get-S3-Data.ipynb) for obtaining SmartSPIM samples from the AIND AWS S3 bucket.

In [1]:
import itertools

import itk
import zarr
import ome_zarr
import numpy as np
import itkwidgets

In [2]:
DO_WRITE_TO_DISK = False
DO_VISUALIZE = True

## Load Data

In [3]:
# http://download.alleninstitute.org/informatics-archive/converted_mouse_ccf/average_template/
average_template = itk.imread('data/CCFv31/average_template_25.nii.gz', pixel_type=itk.F)

# Note: 3.1 template is in mm (3.0 was um)
print(average_template.GetSpacing())

itkVectorD3 ([0.025, 0.025, 0.025])


See [Get-S3-Data](Get-S3-Data.ipynb) for fetching SmartSPIM samples and converting to NIFTI format.

In [4]:
SAMPLE_NAME = 'Ex_647_Em_690'
smartspim_image = itk.imread(rf'data\input\{SAMPLE_NAME}.mhd', pixel_type=itk.F)

if itk.spacing(smartspim_image)[0] > 10.0:
    # Convert from um to mm for common units
    smartspim_image.SetSpacing(itk.spacing(smartspim_image) * 1e-3)

print(type(smartspim_image))
print(itk.spacing(smartspim_image))
print(itk.size(smartspim_image))

<class 'itk.itkImagePython.itkImageF3'>
itkVectorD3 ([0.0288, 0.02898, 0.032])
itkSize3 ([462, 640, 262])


In [5]:
#itkwidgets.view(smartspim_image)

## Validate Data

We expect the data to have roughly the same extent.

In [6]:
def print_extent(image):
    image_size = itk.size(image)
    for index in [[0,0,0], image_size]:
        print(image.TransformIndexToPhysicalPoint(index))

fixed_image_size = itk.size(average_template)
print(f'CCF extent:')
print_extent(average_template)
print(f'SmartSPIM extent:')
print_extent(smartspim_image)

CCF extent:
itkPointD3 ([0, 0, 0])
itkPointD3 ([-11.4, 13.2, -8])
SmartSPIM extent:
itkPointD3 ([0, 0, 0])
itkPointD3 ([-13.3056, -18.5472, -8.384])


## Initialize Registration with `itk`

In [7]:
itk.auto_progress(1)
itk.CenteredTransformInitializer
itk.auto_progress(0)

Loading ITKMesh... Loading ITKMesh... Loading ITKImageFunction... Loading ITKImageFunction... Loading ITKSpatialObjects... Loading ITKSpatialObjects... Loading ITKImageSources... Loading ITKImageSources... Loading ITKImageGrid... Loading ITKImageGrid... Loading ITKFFT... Loading ITKImageCompose... Loading ITKImageCompose... Loading ITKImageStatistics... Loading ITKImageStatistics... Loading ITKPath... Loading ITKPath... Loading ITKImageIntensity... Loading ITKImageIntensity... Loading ITKThresholding... Loading ITKThresholding... Loading ITKConvolution... Loading ITKConvolution... Loading ITKSmoothing... Loading ITKSmoothing... Loading ITKImageGradient... Loading ITKImageGradient... Loading Strain... Loading Strain... Loading SplitComponents... Loading SplitComponents... Loading MeshToPolyData... Loading MeshToPolyData... Loading ITKOptimizers... Loading ITKOptimizers... Loading ITKImageFeature... Loading ITKImageFeature... Loading ITKFiniteDifference... Loading ITKFiniteDifference... 

In [8]:
# Use moments-based initialization to get transform roughly positioning
# sample data on top of CCF data

init_transform = itk.VersorRigid3DTransform[itk.D].New()
init_transform.SetIdentity()

transform_initializer = itk.CenteredTransformInitializer[init_transform, type(average_template), type(smartspim_image)].New()
transform_initializer.SetFixedImage(average_template)
transform_initializer.SetMovingImage(smartspim_image)
transform_initializer.SetTransform(init_transform)
#transform_initializer.MomentsOn()
transform_initializer.GeometryOn()

transform_initializer.InitializeTransform()
print(init_transform)

VersorRigid3DTransform (000001559C985E50)
  RTTI typeinfo:   class itk::VersorRigid3DTransform<double>
  Reference Count: 2
  Modified Time: 1006
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    1 0 0 
    0 1 0 
    0 0 1 
  Offset: [-0.9509, -15.8466, -0.1885]
  Center: [-5.6875, 6.5875, -3.9875]
  Translation: [-0.9509, -15.8466, -0.1885]
  Inverse: 
    1 0 0 
    0 1 0 
    0 0 1 
  Singular: 0
  Versor: [ 0, 0, 0, 1 ]



In [9]:
# Resample sample data into estimated CCF space
# TODO: Use original image extent, but spatially aligned with CCF

# TODO automate transpose via vector-matrix mul
# reordering is a consequence of CCF dir matrix
def to_output(input_vec:list) -> list:
    return [input_vec[1], input_vec[2], input_vec[0]]    

    
output_size=[el * 2 for el in to_output(itk.size(smartspim_image))]
output_spacing=to_output(itk.spacing(smartspim_image))

input_side_lengths = to_output(smartspim_image.TransformIndexToPhysicalPoint(itk.size(smartspim_image)) - smartspim_image.TransformIndexToPhysicalPoint([0,0,0]))
input_side_lengths[1] *= -1
output_origin=[itk.origin(average_template)[dim] - input_side_lengths[dim] * 0.5 for dim in range(3)]

smartspim_image_init = itk.resample_image_filter(smartspim_image,
                                                 transform=init_transform,
                                                 use_reference_image=False,
                                                 size=output_size,
                                                 output_spacing=output_spacing,
                                                 output_direction=average_template.GetDirection(),
                                                 output_origin=output_origin
                                                 )
                                                 #reference_image=average_template)

In [10]:
print('Orig')
print_extent(smartspim_image)
print('Init')
print_extent(smartspim_image_init)
print('Target')
print_extent(average_template)

Orig
itkPointD3 ([0, 0, 0])
itkPointD3 ([-13.3056, -18.5472, -8.384])
Init
itkPointD3 ([9.2736, -4.192, 6.6528])
itkPointD3 ([-17.3376, 32.9024, -10.1152])
Target
itkPointD3 ([0, 0, 0])
itkPointD3 ([-11.4, 13.2, -8])


In [11]:
#itkwidgets.view(smartspim_image_init)
itkwidgets.compare(smartspim_image_init,average_template)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

In [ ]:
print('Orig')
print_extent(smartspim_image)
print('Init')
print_extent(smartspim_image_init)
print('Target')
print_extent(average_template)

In [ ]:
if DO_WRITE_TO_DISK:
    itk.auto_progress(1)
    itk.transformwrite([init_transform], f'data/output/{SAMPLE_NAME}/init_transform.txt', compression=True)
    itk.auto_progress(0)

In [ ]:
if DO_WRITE_TO_DISK:
    itk.imwrite(smartspim_image_init, f'data/output/{SAMPLE_NAME}/smartspim_init.nii.gz', compression=True)

## Register with `itk-elastix`

In [ ]:
itk.auto_progress(1)
itk.ElastixRegistrationMethod
itk.auto_progress(0)

In [ ]:
rigid_parameter_object = itk.ParameterObject.New()
rigid_parameter_object.AddParameterMap(rigid_parameter_object.GetDefaultParameterMap('rigid'))
rigid_parameter_object.AddParameterMap(rigid_parameter_object.GetDefaultParameterMap('affine'))

bspline_map = rigid_parameter_object.GetDefaultParameterMap('bspline')
bspline_map['FinalGridSpacingInPhysicalUnits'] = ('0.5000',)
rigid_parameter_object.AddParameterMap(bspline_map)

print(rigid_parameter_object)

In [17]:
registration_method = itk.ElastixRegistrationMethod[type(average_template), type(smartspim_image)].New(
    fixed_image=average_template,
    moving_image=smartspim_image_init,
    parameter_object=rigid_parameter_object,
    log_to_console=True
)

In [ ]:
# Run registration with `itk-elastix` (will take a few minutes)
registration_method.Update()

In [12]:
image = itk.imread(f'data/output/{SAMPLE_NAME}/{SAMPLE_NAME}_registered_grid_density_0_5.nii.gz')
itkwidgets.checkerboard(image, average_template)

In [ ]:
itkwidgets.checkerboard(registration_method.GetOutput(), average_template)

In [22]:
if DO_WRITE_TO_DISK:
    itk.imwrite(registration_method.GetOutput(), f'data/output/{SAMPLE_NAME}/{SAMPLE_NAME}_registered_grid_density_0_5.nii.gz', compression=True)

    for index in range(rigid_parameter_object.GetNumberOfParameterMaps()):
        registration_method.GetTransformParameterObject().WriteParameterFile(
            registration_method.GetTransformParameterObject().GetParameterMap(index), f'data/output/{SAMPLE_NAME}/elastix-transform{index}.txt')